# Nature NIH conflict of interest investigation

Working process to determine the percentage of grants given to each institution that have a FCOI (financial conflict of interest) investigation associated with them. 

`43736 Reardon with redactions.xls`: NIH FOI document listing all conflict of interest reports that investigators filed in 2012 and 2013

**NIH Research Portfolio Online Reporting Tools (RePORT)** - A repository of NIH-funded research projects and access publications and patents resulting from that funding.

**ExPORTER Data Catalog** [http://exporter.nih.gov/ExPORTER_Catalog.aspx](http://exporter.nih.gov/ExPORTER_Catalog.aspx) - makes downloadable versions of the data accessed through the RePORT Expenditures and Results (RePORTER) interface available to the public. 

## Download and unzip NIH Reporter files relating to January 2013 to May 2015

Download the relevant files

In [1]:
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_029.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_028.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_027.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_026.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_025.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_024.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_023.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_022.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_021.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_020.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_019.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_018.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_017.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_016.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_015.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_014.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_013.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_012.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_011.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_010.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_009.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_008.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_007.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_006.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_005.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_004.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_003.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_002.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2015_001.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2014.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2013.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  235k  100  235k    0     0   365k      0 --:--:-- --:--:-- --:--:--  365k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  513k  100  513k    0     0   240k      0  0:00:02  0:00:02 --:--:--  240k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  376k  100  376k    0     0   204k      0  0:00:01  0:00:01 --:--:--  203k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2973k  100 2973k    0     0   480k      0  0:00:06  0:00:06 --:--:--  554k
  % Total    % Received % Xferd  Average Speed   Tim

Unzip the downloaded files

In [2]:
!for f in *.zip; do unzip "${f}"; done;

Archive:  RePORTER_PRJ_C_FY2013.zip
  inflating: RePORTER_PRJ_C_FY2013.csv  
Archive:  RePORTER_PRJ_C_FY2014.zip
  inflating: RePORTER_PRJ_C_FY2014.csv  
Archive:  RePORTER_PRJ_C_FY2015_001.zip
  inflating: RePORTER_PRJ_C_FY2015_001.csv  
Archive:  RePORTER_PRJ_C_FY2015_002.zip
  inflating: RePORTER_PRJ_C_FY2015_002.csv  
Archive:  RePORTER_PRJ_C_FY2015_003.zip
  inflating: RePORTER_PRJ_C_FY2015_003.csv  
Archive:  RePORTER_PRJ_C_FY2015_004.zip
  inflating: RePORTER_PRJ_C_FY2015_004.csv  
Archive:  RePORTER_PRJ_C_FY2015_005.zip
  inflating: RePORTER_PRJ_C_FY2015_005.csv  
Archive:  RePORTER_PRJ_C_FY2015_006.zip
  inflating: RePORTER_PRJ_C_FY2015_006.csv  
Archive:  RePORTER_PRJ_C_FY2015_007.zip
  inflating: RePORTER_PRJ_C_FY2015_007.csv  
Archive:  RePORTER_PRJ_C_FY2015_008.zip
  inflating: RePORTER_PRJ_C_FY2015_008.csv  
Archive:  RePORTER_PRJ_C_FY2015_009.zip
  inflating: RePORTER_PRJ_C_FY2015_009.csv  
Archive:  RePORTER_PRJ_C_FY2015_010.zip
  inflating: RePORTER_PRJ_C_FY2015_010.cs

Delete the archives, leaving just the csv files.

In [3]:
! for f in *.zip; do rm "${f}"; done;

We should now have 31 CSV files.

## Clean and combine the NIH reporter documents

Convert the CSV files from `iso-8859-1` to `utf-8`

In [2]:
! for f in *.csv; do in2csv -e iso-8859-1 -f csv "${f}" > temp && mv temp "${f}"; done;

Combine all of the files into one CSV file:

In [3]:
!csvstack RePORTER_PRJ_C_FY2015_029.csv RePORTER_PRJ_C_FY2015_028.csv RePORTER_PRJ_C_FY2015_027.csv RePORTER_PRJ_C_FY2015_026.csv RePORTER_PRJ_C_FY2015_025.csv RePORTER_PRJ_C_FY2015_024.csv RePORTER_PRJ_C_FY2015_023.csv RePORTER_PRJ_C_FY2015_022.csv RePORTER_PRJ_C_FY2015_021.csv RePORTER_PRJ_C_FY2015_020.csv RePORTER_PRJ_C_FY2015_019.csv RePORTER_PRJ_C_FY2015_018.csv RePORTER_PRJ_C_FY2015_017.csv RePORTER_PRJ_C_FY2015_016.csv RePORTER_PRJ_C_FY2015_015.csv RePORTER_PRJ_C_FY2015_014.csv RePORTER_PRJ_C_FY2015_013.csv RePORTER_PRJ_C_FY2015_012.csv RePORTER_PRJ_C_FY2015_011.csv RePORTER_PRJ_C_FY2015_010.csv RePORTER_PRJ_C_FY2015_009.csv RePORTER_PRJ_C_FY2015_008.csv RePORTER_PRJ_C_FY2015_007.csv RePORTER_PRJ_C_FY2015_006.csv RePORTER_PRJ_C_FY2015_005.csv RePORTER_PRJ_C_FY2015_004.csv RePORTER_PRJ_C_FY2015_003.csv RePORTER_PRJ_C_FY2015_002.csv RePORTER_PRJ_C_FY2015_001.csv RePORTER_PRJ_C_FY2014.csv RePORTER_PRJ_C_FY2013.csv > csv/nih_all.csv

Remove the individual csv files

In [5]:
! for f in *.csv; do rm "${f}"; done;

Count the number of lines in the new file

In [4]:
!wc -l csv/nih_all.csv #Count the number of lines in the file

  180715 csv/nih_all.csv


Extract just the relevant columns from the four NIH documents into new documents and Convert `iso-8859-1` to `utf-8`.

In [6]:
!csvcut -c 'APPLICATION_ID, PROJECT_TITLE, PI_NAMEs, ORG_NAME, FUNDING_MECHANISM, FY, PROJECT_START, PROJECT_END, TOTAL_COST' csv/nih_all.csv > temp && mv temp csv/nih_all.csv

In [7]:
!csvcut -n csv/nih_all.csv #Get a numbered list of the columns

  1: APPLICATION_ID
  2: PROJECT_TITLE
  3: PI_NAMEs
  4: ORG_NAME
  5: FUNDING_MECHANISM
  6: FY
  7: PROJECT_START
  8: PROJECT_END
  9: TOTAL_COST


The regulation does not apply to Phase I Small Business Innovation Research (SBIR) and Small Business Technology Transfer (STTR) applications/awards so remove all `SBIR-STTR` entires from the `FUNDING_MECHANISM` column.

In [8]:
!csvgrep -c 5 -r "SBIR-STTR" -i csv/nih_all.csv > csv/temp.csv && mv csv/temp.csv csv/nih_all.csv

Count the number of lines remaining in the file nb. should be less than previous count

In [9]:
!wc -l csv/nih_all.csv #Count the number of lines in the file

  176787 csv/nih_all.csv


## Begin examining the data in Python 

Load pandas, pandasql and csvkit

In [11]:
import pandas
import pandasql
import csvkit

Load the combined NIH csv into a pandas dataframe

In [18]:
nih_df = pandas.read_csv("./csv/nih_all.csv")
nih_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)

nih_org_series = nih_df['org_name'].value_counts()

nih_org_df = pandas.DataFrame(nih_org_series, columns=['total-grants'])

We now have a data frame with a total number of grants pre institution

In [19]:
# Count of grants per institution
nih_org_df.head()

,total-grants
JOHNS HOPKINS UNIVERSITY,3828
"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",3680
UNIVERSITY OF PENNSYLVANIA,3429
UNIVERSITY OF MICHIGAN,3046
UNIV OF NORTH CAROLINA CHAPEL HILL,2963


## Load the FCOI data into a pandas dataframe

Load in the NIH fcoi data received via a freedomo of information request

In [20]:
!in2csv --sheet "SQL Results" "ref/43736 Reardon with redactions.xls" > csv/fcoi.csv

In [21]:
!csvcut -n csv/fcoi.csv #Get a numbered list of the columns

  1: FCOI_ID
  2: PROJECT NUMBER
  3: PROJECT TITLE
  4: CONTACT PD/PI NAME
  5: FCOI INVESTIGATOR NAME
  6: SUBMITTING INSTITUTION
  7: FCOI REPORT TYPE
  8: FCOI AMOUNT VALUE
  9: FCOI REPORT STATUS


In [22]:
fcoi_df = pandas.read_csv("./csv/fcoi.csv")
fcoi_df.rename(columns = lambda x: x.replace(' ','_').lower(),  inplace=True)

fcoi_org_series = fcoi_df['submitting_institution'].value_counts()
fcoi_org_df = pandas.DataFrame(fcoi_org_series, columns=['total-fcoi-claims'])

In [23]:
# Count of total fcoi claims per institution
fcoi_org_df.head(4)

,total-fcoi-claims
YALE UNIVERSITY,123
UNIVERSITY OF WISCONSIN-MADISON,106
UNIV OF NORTH CAROLINA CHAPEL HILL,88
MASSACHUSETTS GENERAL HOSPITAL,86


## Join the NIH and FCOI data frames for comparison

Combine the total counts for number of grants and number of fcoi claims per institution and print this out to a csv flie called "institution_fcoi_nih.csv"

In [24]:
# pd.merge(left_frame, right_frame, on='key', how='inner')
org_merge = pandas.merge(fcoi_org_df, nih_org_df, left_index=True, right_index=True, how='inner')

org_merge.to_csv("./csv/institution_fcoi_nih.csv")

org_merge.head()

,total-fcoi-claims,total-grants
JOHNS HOPKINS UNIVERSITY,66,3828
"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",2,3680
UNIVERSITY OF PENNSYLVANIA,65,3429
UNIVERSITY OF MICHIGAN,39,3046
UNIV OF NORTH CAROLINA CHAPEL HILL,88,2963
